## SQL 数据清洗（Data cleaning）
这篇文章主要讲一下数据清洗，相对于这个系列之前的内容，这章更偏向于实际应用了。有点激动了有没有。
在这个章节中，我们将了解和学习一些技巧用来：

*清理和重组混乱的数据。

*将列转换为不同的数据类型。

*操作null

这些技巧可以帮助你从原始数据获取对分析有用的干净数据。


## 1
首先介绍3个新的语句

* LEFT
* RIGHT
* LENGTH

LEFT从指定列的开头(或从左边)开始为每一行提取指定数量的字符。正如您在这里看到的，可以使用LEFT(phone_number, 3)提取电话号码的前三位数字。

RIGHT从末尾(或从右)开始为指定列中的每一行提取指定数量的字符。正如您在这里看到的，您可以使用RIGHT(phone_number, 8)提取电话号码的最后8位数字。

LENGTH提供指定列的每一行的字符数。在这里，您看到我们可以使用它来获得每个电话号码的长度为length (phone_number)。


## 练习 1. 计算一下accounts表中有多少网站类型

要回答这个问题，你需要找到accounts 表中每个account的website的domain的类型。就是我们常见的.com, .org 等等

In [20]:
import pandas as pd
import sqlite3 as sql

database = 'parchposey.db'
connection = sql.connect(database)

In [22]:
query = " SELECT RIGHT(website, 3) AS domain, COUNT(*) num_companies \
            FROM accounts \
            GROUP BY domain \
            ORDER BY num_companies DESC;"

print(query)

 SELECT RIGHT(website, 3) AS domain, COUNT(*) num_companies             FROM accounts             GROUP BY domain             ORDER BY num_companies DESC;


In [26]:
query = " SELECT RIGHT(website AS domain,\
                 COUNT(*) AS num_companies \
          FROM accounts \
          GROUP BY domain \
          ORDER BY num_companies DESC;"

df = pd.read_sql(query, connection)
df

,domain,num_companies
0,www.yum.com,1
1,www.xerox.com,1
2,www.xcelenergy.com,1
3,www.wnr.com,1
4,www.wm.com,1
...,...,...
346,www.abbott.com,1
347,www.aa.com,1
348,www.3m.com,1
349,www.21cf.com,1


## 2
之后是常用于字符串操作的另外4个语句

* POSITION
* STRPOS
* SUBSTR
* LOWER
* UPPER

POSITION接受一个字符和一个列，并为每一行提供字符所在位置的索引。在SQL中，第一个位置的索引是1。如果您来自另一种编程语言，许多人从0开始索引。在这里，您可以将逗号的索引作为POSITION('，' IN city_state)提取出来。

STRPOS提供了与POSITION相同的结果，但是实现这些结果的语法略有不同，如下所示:STRPOS(city_state， '，')。

注意，POSITION和STRPOS都是区分大小写的，所以查找A和查找A是不同的。

SUBSTR(列名, 开始位置, 截取几个字符) 可以在字符串的任意位置截取任意长度的字符
例如我想获得Date:‘24/03/1994’的年份，那我就可以使用如下语句： 
SUBSTR（Date, 7, 4）

因此，如果您希望提取索引，而不管字母的大小写，您可能希望使用LOWER或UPPER使所有字符小写或大写。

## 3
之后是常用于字符串拼接的两个语句

* CONCAT
* Piping ||

这两种方法都可以跨行组合字符串，例如

CONCAT(first_name， ' '， last_name)或使用管道作为first_name || ' ' || last_name。

## 4

之后是常用于操作时间的三个语句

* TO_DATE
* CAST
* Casting with ::

在这里，DATE_PART('month'， TO_DATE(month， 'month'))将一个月的名称更改为与该特定的月份相关联的数字。

然后可以使用CAST将字符串更改为日期。CAST实际上对更改许多列类型很有用。通常，您可能会像这里看到的那样，使用CAST(date_column as date)将字符串更改为日期。但是，您可能希望根据数据类型对列进行其他更改。你可以在这里看到其他的例子。



## 5


针对NULLs 也有一个相应的处理函数 COALESCE。

之前我们有提到过很多聚合函数在计算的过程中是不会考虑NULL的，这就导致当表中有NULLS时这些聚合函数的结果某种程度上不是十分的准确，COALESCE 可以将NULLS转换为特定的数据进而使得聚合函数的结果可以更加的准确。

下面通过一个练习来了解一下 COALESCE 应该如何使用。

### step 1
orders 和 accounts 联合表中有缺失的数据，首先同过查询来找到这条数据。

In [6]:
query = " SELECT * \
    FROM accounts a \
    LEFT JOIN orders o \
    ON a.id = o.account_id \
    WHERE o.total IS NULL;"

df = pd.read_sql(query, connection)
df

,id,name,website,lat,long,primary_poc,sales_rep_id,id,account_id,occurred_at,standard_qty,gloss_qty,poster_qty,total,standard_amt_usd,gloss_amt_usd,poster_amt_usd,total_amt_usd
0,1731,Goldman Sachs Group,www.gs.com,40.757444,-73.967309,Loris Manfredi,321690,None,None,None,None,None,None,None,None,None,None,None


这里我肯可以发现orders和accounts的表中，有一行数据是不完整的

### step 2

基本语法 COALESCE(a, b) AS c, 其中a是一列数据，b可以是一列数据或者是一个固定的值，如果a中的数据是NULL的话，就会被替换成为b 

首先通过COALESCE来使用accouts.id 来填充orders.id中的NULLs

COALESCE(o.id, a.id) filled_id

其次使用accounts.id来填充orders表的account_id列

COALESCE(o.account_id, a.id) account_id


In [14]:
query = " SELECT COALESCE(o.id, a.id) filled_id, a.name, a.website, a.lat, a.long, a.primary_poc, a.sales_rep_id, COALESCE(o.account_id, a.id) account_id, o.occurred_at, o.standard_qty, o.gloss_qty, o.poster_qty, o.total, o.standard_amt_usd, o.gloss_amt_usd, o.poster_amt_usd, o.total_amt_usd \
            FROM accounts a \
            LEFT JOIN orders o \
            ON a.id = o.account_id \
            WHERE o.total IS NULL; "

df = pd.read_sql(query, connection)
df

,filled_id,name,website,lat,long,primary_poc,sales_rep_id,account_id,occurred_at,standard_qty,gloss_qty,poster_qty,total,standard_amt_usd,gloss_amt_usd,poster_amt_usd,total_amt_usd
0,1731,Goldman Sachs Group,www.gs.com,40.757444,-73.967309,Loris Manfredi,321690,1731,None,None,None,None,None,None,None,None,None


### Step 3

将空白的qty和usd的列用0来填充


In [13]:
query = " SELECT COALESCE(o.id, a.id) filled_id, a.name, a.website, a.lat, a.long, a.primary_poc, a.sales_rep_id, COALESCE(o.account_id, a.id) account_id, o.occurred_at, COALESCE(o.standard_qty, 0) standard_qty, COALESCE(o.gloss_qty,0) gloss_qty, COALESCE(o.poster_qty,0) poster_qty, COALESCE(o.total,0) total, COALESCE(o.standard_amt_usd,0) standard_amt_usd, COALESCE(o.gloss_amt_usd,0) gloss_amt_usd, COALESCE(o.poster_amt_usd,0) poster_amt_usd, COALESCE(o.total_amt_usd,0) total_amt_usd \
            FROM accounts a \
            LEFT JOIN orders o \
            ON a.id = o.account_id \
            WHERE o.total IS NULL; "

df = pd.read_sql(query, connection)
df

,filled_id,name,website,lat,long,primary_poc,sales_rep_id,account_id,occurred_at,standard_qty,gloss_qty,poster_qty,total,standard_amt_usd,gloss_amt_usd,poster_amt_usd,total_amt_usd
0,1731,Goldman Sachs Group,www.gs.com,40.757444,-73.967309,Loris Manfredi,321690,1731,None,0,0,0,0,0,0,0,0


至此我们就已经简单通过使用COALESCE处理了这行空数据

## 总结
现在有许多工具可以帮助清理SQL中的混乱数据。手动清理数据非常繁琐，但是现在您可以使用新的技能大规模清理数据。

对于任何数据清理功能的提醒，本课中的概念都根据您所学的功能进行标记。如果你一开始对这些功能感到不舒服，那是正常的——这些需要一些时间来适应。不要害怕通过第二遍阅读材料来提高你的技能!

记住所有这些功能并不是必要的，但您确实需要能够遵循文档，并从解决以前的问题中学习如何解决新问题。
